In [2]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations
import itertools
import random
from copy import deepcopy

In [4]:
def is_edge_dominating_set(G, candidate_set):
    candidate_set_normalized = {tuple(sorted(e)) for e in candidate_set}
    all_edges_normalized = {tuple(sorted(e)) for e in G.edges()}

    covered_edges = set()

    for e in candidate_set_normalized:
        covered_edges.add(e)
        for u in [e[0], e[1]]:
            for v in G[u]:
                if u != v:
                    covered_edges.add(tuple(sorted((u, v))))

    return all_edges_normalized <= covered_edges

def brute_force_edge_dominating_set(G):
    all_edges = list(G.edges())
    min_edge_dom_set = None
    min_size = float('inf')

    for i in range(1, len(all_edges) + 1):
        for subset in combinations(all_edges, i):
            if is_edge_dominating_set(G, subset):
                if i < min_size:
                    min_size = i
                    min_edge_dom_set = subset
    return min_edge_dom_set

def greedy_edge_dominating_set(G):
    normalized_edges = {tuple(sorted(e)) for e in G.edges()}
    remaining_edges = set(normalized_edges)
    edge_dominating_set = set()

    while remaining_edges:
        max_cover_edge = max(remaining_edges, key=lambda e: len({tuple(sorted(f)) for f in G.edges(e[0])}.union({tuple(sorted(f)) for f in G.edges(e[1])}) & remaining_edges))
        edge_dominating_set.add(max_cover_edge)
        # print(f"remaining edges before: {remaining_edges}")
        # print("max_cov")
        # print(max_cover_edge)

        covered_edges = {tuple(sorted(f)) for f in G.edges(max_cover_edge[0])}.union({tuple(sorted(f)) for f in G.edges(max_cover_edge[1])})
        remaining_edges.difference_update(covered_edges)
        # print("cov_edg")
        # print(covered_edges)
        # print(f"remaining edges after: {remaining_edges}")
    
    return edge_dominating_set

def convert_binary_to_edges(graph, solution):
    edges = list(graph.edges())
    solution = [edges[i] for i in range(len(edges)) if solution[i] == 1]
    return solution
    
def draw_graph_with_solution(graph, solution, algorithm):
    # pos = nx.spring_layout(graph)
    # pos = nx.circular_layout(graph)
    pos = nx.fruchterman_reingold_layout(graph)
    # pos = nx.bfs_layout(graph, (1,2))

    nx.draw(graph, pos, with_labels=True, node_color='lightblue', node_size=700, edge_color='lightgray')
    
    nx.draw_networkx_edges(graph, pos, edgelist=solution, width=2.0, edge_color='red')

    plt.title(f'Graph with {algorithm} Edge Dominating Set Solution (V: {len(graph.nodes)}, E: {len(graph.edges)})')
    plt.show()


In [5]:
class Individual:
    def __init__(self, graph, chromosome=None, fitness_method=1):
        self.graph = graph
        self.chromosome = chromosome if chromosome is not None else self._random_chromosome()
        self.fitness_method = fitness_method
        self.fitness = self.calculate_fitness()

    def _random_chromosome(self):
        edges = list(self.graph.edges())
        return [random.randint(0, 1) for _ in edges]

    def calculate_fitness(self):
        edges = list(self.graph.edges())
        covered = set()
        adj_list = {node: set(self.graph.adj[node]) for node in self.graph.nodes()}

        for i, include in enumerate(self.chromosome):
            if include:
                edge = edges[i]
                covered.add(frozenset(edge))
                for neighbor in adj_list[edge[0]]:
                    covered.add(frozenset((edge[0], neighbor)))
                for neighbor in adj_list[edge[1]]:
                    covered.add(frozenset((edge[1], neighbor)))

        all_edges = {frozenset(e) for e in edges}
        if self.fitness_method == 1:
            if all_edges <= covered:
                return len(all_edges) - sum(self.chromosome)
            return float('-inf')
        else:
            uncovered_edges = len(all_edges - covered)
            covered_edges = len(all_edges) - uncovered_edges
            fitness_value = covered_edges - sum(self.chromosome) - uncovered_edges

        return fitness_value
    
    def update_fitness(self):
        self.fitness = self.calculate_fitness()

    def __repr__(self):
        return f"Individual(fitness={self.fitness}, chromosome={self.chromosome})"


In [7]:
def initialize_population(graph, size, fitness_method):
    return [Individual(graph, fitness_method=fitness_method) for _ in range(size)]

def select(population, tournament_size):
    tournament = random.sample(population, tournament_size)
    return max(tournament, key=lambda ind: ind.fitness)

def crossover(parent1, parent2, child1, child2):
    point = random.randint(1, len(parent1.chromosome) - 1)
    child1.chromosome = parent1.chromosome[:point] + parent2.chromosome[point:]
    child2.chromosome = parent2.chromosome[:point] + parent1.chromosome[point:]

def mutate(individual, mutation_rate=0.05):
    for i in range(len(individual.chromosome)):
        if random.random() < mutation_rate:
            individual.chromosome[i] = 1 - individual.chromosome[i]
    individual.update_fitness()

def linear_decay_mutation_rate(initial_rate, final_rate, current_generation, max_generations):
    return initial_rate - (initial_rate - final_rate) * (current_generation / max_generations)
    
def genetic_algorithm(graph, population_size=100, generations=10, initial_mutation_rate=0.05, elitism_percent=0.05, gradual_mutation=False, fitness_method=1):
    population = initialize_population(graph, population_size, fitness_method)
    new_population = deepcopy(population)
    mutation_rate = initial_mutation_rate
    final_mutation_rate = initial_mutation_rate / 5
    elitism_count = int(population_size * elitism_percent)
    if elitism_count % 2 == 1:
        elitism_count += 1
    for generation in range(generations):
        
        if gradual_mutation:
            mutation_rate = linear_decay_mutation_rate(initial_mutation_rate, final_mutation_rate, generation, generations)
        
        population = sorted(population, key=lambda ind: ind.fitness, reverse=True)
        new_population[:elitism_count] = population[:elitism_count]

        for i in range(elitism_count, population_size, 2):
            parent1 = select(population, population_size // 10)
            parent2 = select(population, population_size // 10)
            crossover(parent1, parent2, new_population[i], new_population[i+1])
            mutate(new_population[i], mutation_rate)
            mutate(new_population[i+1], mutation_rate)

        population[:] = new_population[:]

    best_individual = max(population, key=lambda ind: ind.fitness)
    return best_individual.chromosome

def maximal_matching_eds(graph):
    matching = nx.maximal_matching(graph)
   
    eds = set(matching)
    
    return eds
